# EDS Test Harness
This notebook can be used to test the EDS web services. It is interactive and users can modify it as they see fit. Modifcations to this notebook will be available to the user while they are actively using the notebook, they are not saved for future sessions.

Instructions

**1: Setup**
* Verify the value of the variable client_key is the provided key: saropsproposal
* Use the Run button above to set the key

**2: Get Data Catalog**
* Run the 'Get Data Catalog' box by pressing the Run button again

**3: Request Configuration**
* Edit the variables in the 'Get Data Request Configuration' box to specify an EDS data request. Use the results from the 'Get Data Catalog' request to set the 'dataset_id' and 'source' variables. The source is the header text for each dataset. The 'Get Data Request' box contains a default request for HYCOM_GLOBAL data that can be run as an example
* Run the 'Get Data Request Configuration' Box

**4: Get Data Request**
* Run the 'Get Data Request' box. This code will make a GetData request to EDS. When the file name is returned, it then makes a GetStatus request to EDS until a status of 'COMPLETE' is returned. When GetStatus returns 'COMPLETE', a Downloadfile request is made and a netCDF file is downloaded to the cloud machine running this notebook. A link will be displayed to the user that they can click to download the file from the cloud to their local machine.
* Click on the link to download the netCDF file to your local machine

In [ ]:
# Test Configuration Box

# Set the client key to use
client_key = "change_me_to_provided_key"



In [ ]:
# Get Data Catalog Box
# Gets the Data Catalog from EDS 
import zeep
import xml.etree.ElementTree as ET 
import datetime
eds_wsdl = "http://coastmap.com/eds20/eds.asmx?wsdl"
client = zeep.Client(wsdl=eds_wsdl)

# get the catalog from EDS
get_catalog = client.service.GetDataCoverage(clientKey=client_key)
root = ET.fromstring(get_catalog) 

print('Datasets available for client_key:', client_key)
# create empty list for news items 
for item in root.findall('dataset'):
    print(item.attrib['source'])
    print("   ID:          ",item.attrib['id'])
    print("   Name:        ",item.attrib['shortname'])
    print("   Description: ",item.attrib['description'])
    print("   Time span:   ",item.attrib['starttime'],item.attrib['endtime'])
    print("   Lon Extents: ",item.attrib['extentleft'],item.attrib['extentright'])
    print("   Lat Extents: ",item.attrib['extenttop'],item.attrib['extentbottom'])


In [ ]:
# Get Data Request Configuration Box
# Define the request - Change values
# Get the ID and source from the data catalog results above
dataset_ID = 696 
source = "HYCOM_GLOBAL"
lon_min = -80.5
lon_max = -80
lat_min = 30.5
lat_max = 40.0
start_time = datetime.datetime(2020,1,12,6,0,0)
end_time = datetime.datetime(2020,1,14,3,0,0)



In [ ]:
# Get Data Request Box
# Make a request to EDS to download the data for the request 
from time import sleep
import os
from IPython.display import FileLink, FileLinks
request_id = client.service.GetData(Key=client_key,
                                    SourceID=dataset_ID,
                                    SourceName=source,
                                    OutputType=1,
                                    StartDate=start_time,
                                    EndDate=end_time,
                                    X1=lon_min,
                                    X2=lon_max,
                                    Y1=lat_min,
                                    Y2=lat_max,
                                    Zipped=True)
print(request_id)
status = None
while status is None or 'IN_PROCESS' in status:
    sleep(2)
    status = client.service.GetStatus(request_id)
    if status is not None:
        print('status= '+ status)
        if 'COMPLETE' in status:
            print('start download')
            download_suc = client.service.DownloadFile(request_id)
            nc_path = os.path.join(".", request_id)
            with open(nc_path,'wb') as f:
                    f.write(download_suc)
                    
            
            print("Click the link to download the file to your local machine")
            display(FileLink(nc_path))
            